# テーマ設定

課題番号**004**の**文章をカテゴリー分類するモデルの作成**に取り組みました。

データセットとして**ライブドアニュースコーパス**を使用し、単語分散表現・GRUとCNN、Attentionを組み合わせてより良いモデルを作成しました。

# Abstract

本プロダクト開発課題では、ライブドアニュースコーパスを用いた日本語文章分類の精度向上を目的としてモデルの構築を行いました。9つのカテゴリへの多クラス分類タスクに対し、2つの深層学習モデルを構築し、その性能を比較検証しました。

まず、ベースラインとして、局所的な特徴抽出に優れたCNN（畳み込みニューラルネットワーク）と、時系列情報を捉えるGRUを組み合わせたモデルを実装しました。

次に、改善モデルとして、Transformerの設計思想に基づき、CNNを自己注意機構（Self-Attention）に置き換え、文全体の構造的・文脈的関係性を捉えるモデルを構築しました。両モデルの実装にはPyTorchを用い、テキストの前処理にはJanomeによる形態素解析を適用しました。

実験の結果、ベースラインモデルは正解率83.3%を達成しました。一方、残差接続や層正規化を取り入れた改善モデルは、正解率を89.0%まで向上させ、約5.7ポイントの大幅な性能向上を確認しました。特に、ベースラインが苦手としていた広範なテーマのカテゴリ（例: livedoor-homme, peachy）において、F1スコアの顕著な改善を確認しました（それぞれ0.50→0.65, 0.68→0.77）。

以上の結果から、局所的な特徴だけでなく文全体の文脈情報を捉える自己注意機構が、本タスクにおいてCNNよりも優れた特徴抽出器として機能し、分類精度向上に有効であるという結論が得られました。

## 環境構築

### ライブラリのインストール
必要なライブラリをインストールし、実行環境のバージョンを統一します。

In [ ]:
import sys

# Google colab環境であるか判定
if "google.colab" in sys.modules:
    # ライブラリのインストール
    %pip install --no-warn-conflicts torch==2.1.1 torchvision==0.16.1 nltk==3.8.1 janome==0.5.0 numpy
else:
    print("Not Google Colab")

### ドライブのマウント

In [ ]:
# Google colab環境であるか判定
if "google.colab" in sys.modules:
    # マウントを行う
    from google.colab import drive

    drive.mount("/content/drive")
else:
    print("Not Google Colab")

### ライブラリのインポート

In [2]:
import copy
import io
import os
import math
import re
import tarfile
import time
import urllib.request
from typing import Optional

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pickle
from tqdm.auto import tqdm

from janome.tokenizer import Tokenizer
from nltk import tokenize
from sklearn.model_selection import train_test_split

# データ収集

## データセットの準備
コーディング試験Chapter11-2で使用したLivedoorニュースコーパスをダウンロードして使用します。
インターネット上に公開されているデータセットを以下のコードでダウンロードします。

In [ ]:
with urllib.request.urlopen("https://www.rondhuit.com/download/ldcc-20140209.tar.gz") as res:
    with open("ldcc-20140209.tar.gz", "wb") as f:
        f.write(res.read())

ダウンロードしたファイルは圧縮されているので、作業フォルダに展開します。

In [ ]:
# Pathの設定
# Google colab環境であるか判定
if "google.colab" in sys.modules:
    # マイドライブ内のデータを読み込むpathに設定
    livedoor_path = "/content/drive/MyDrive/product_assignment/ldcc-20140209.tar.gz"
else:
    livedoor_path = "ldcc-20140209.tar.gz"

DATA_PATH = "/content/drive/MyDrive/product_assignment/"
tar = tarfile.open(livedoor_path)
tar.extractall(DATA_PATH)
tar.close()

## データセットの作成
カテゴリをラベル、ファイル内の文章をデータとしてそれらが対になったデータをCSV形式にして保存します。

In [ ]:
from tqdm.auto import tqdm # 進捗バーを表示

# 展開したテキスト群が置かれている親ディレクトリ
DATA_DIR = '/content/drive/MyDrive/product_assignment/text/'

# カテゴリ名（サブディレクトリ）のリストを取得
# 不要なファイル（例：LICENSE.txt）は除外
categories = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
print("対象カテゴリ：", categories)

# 最終的にDataFrameにするための、行データ（辞書）を格納するためのリスト
all_data = []

# tqdmを使って進捗を表示しながらカテゴリごとにループ
for category in tqdm(categories, desc="カテゴリ処理中"):
    category_path = os.path.join(data_dir, category)

    files = os.listdir(category_path)
    for file_name in files:
        # category内のREADME.mdはスキップ
        if file_name.endwith(".txt"):
            file_path = os.path.join(category_path, file_name)

            try: 
                with open(file_path, 'r', encoding='utf-8') as f:
                    # 最初の2行はURLとタイムスタンプなので読み飛ばし、3行目以降を本文として取得
                    lines = f.readlines()
                    text_body = "".join(lines[2:]).strip()

                    # ラベル（カテゴリ名）とテキスト本文を辞書としてリストに追加
                    all_data.append( {'label': category, 'text': text_body} )
            except Exception as e:
                print(f"Error readin {file_path}: {e}")

# ループ完了後、リストから一気にDataFrameを作成
df = pd.DataFrame(all_data)

# 作成したDataFrameをCSVとして保存（インデックスは不要なのでindexにはFalseを設定）
df.to_csv(data_dir + "livedoor_news_corpus.csv", index=False, encoding="utf-8-sig")

print("\nCSVファイルの作成が完了しました。")
print("データ件数：", len(df))
print("Head：\n", df.head())

## 言語データの前処理

日本語を形態素解析して単語表層形に分かち書きします。

そのうえで単語をIDに変換します。`"CUDA out of memory"` の回避のために文章が512文字を超えた場合には切り詰めを行いました。

#### 日本語の分かち書きメソッド

In [ ]:
wakati = Tokenizer()

""" 日本語のトークン化 """
def tokenize_ja(sentences_list):
    wakati_list = []
    print("トークン処理を開始します。")
    for sentence in tqdm(sentences_list):
        # tokenizeから返される表層形を分かち書きリストに登録
        wakati_list.append([item.surface for item in wakati.tokenize(sentence)])
    return wakati_list

#### 単語からIDへの辞書を生成

In [ ]:
""" 単語からIDへの辞書を作成 """
def create_word_id_dict(sentences):
    word_to_id = {}  # 単語からIDへの変換辞書
    id_to_word = {}  # IDから単語への逆引き辞書
    # 0はパディング／未知語用に予約
    word_to_id['<PAD>/<UNK>'] = 0
    id_to_word[0] = '<PAD>/<UNK>'

    # すべての文章をループ  
    for sentence in sentences:
        # 文章内の各単語をループ
        for word in sentence:
            # もし単語がまだ辞書に登録されていなければ、新しいIDを割り振る、
            if word not in word_to_id:
                # 新しいIDとして、現在の辞書のサイズ（登録済みの単語数）を使用する
                tmp_id = len(word_to_id)
                word_to_id[word] = tmp_id
                id_to_word[tmp_id] = word

    # (単語をキー、IDをバリューとする辞書, IDをキー、単語をバリューとする辞書)のタプルを返す
    return word_to_id, id_to_word

#### 文章をID列に変換

In [ ]:
""" 単語で構成された文章のリストを対応するIDのリストに変換 """
def convert_sentences_to_ids(sentences, word_to_id):
    sentence_id_list = []
    for sentence in sentences:
        # dict.get(key, default)メソッドによって、未知語でもエラーにならずにデフォルトである<UNK>のIDを返す
        sentence_ids = [word_to_id.get(word, 0) for word in sentence]
        sentence_id_list.append( sentence_ids )

    # IDに変換された文章のリストを返す 
    return sentence_ids

#### 文章のパディング処理

In [ ]:
""" IDに変換され文章のリスト"に対して、paddingと打ち切り処理を行う """
def padding_and_truncate_sentence(sentences, max_len=512):
    # 処理が行われた後の文章IDを格納するリスト
    processed_sentences = []

    for sentence in sentences:
        # １．打ち切り
        # 文章の長さが max_lenを超える場合、末尾から max_len分だけを取得します。
        # 文章の末尾に重重要な情報含まれる場合が多いため、前から切り捨てます
        # 改善モデルにおけるMultiheadAttention層が内部で行う計算で、
        # 非常に巨大な行列を作成しようとしてGPUのメモリが足りなくなる問題への対応として実施しました。
        sentence = sentence[-max_len:]

        # ２．padding
        # 文章の長さがmax_lenに満たない場合、差分を計算
        padding_size = max_len - len(sentence)

        # 足りない分だけ <PAD>のIDのリストを作成し、文章の前方に連結
        padding = [0] * padding_size
        processed_sentences.append(padding + sentence)

    return processed_sentences

#### 前処理の実行

In [ ]:
# 生のCSVの読み込み
df = pd.read_csv(DATA_PATH + "livedoor_news_corpus.csv")

# 文章カテゴリ（ラベル）をID化
label_to_id = {label: i for i, label in enumerate(df['label'].unique())}
id_to_label = {i: label for i, label in enumerate(df['label'].unique())}

# テキストの分かち書き
ja_sentences = tokenize_ja(df["text"].tolist())

# 単語辞書の作成
word_to_id, id_to_word = create_word_id_dict(ja_sentences)

# 文章をID列に変換
sentence_ids = convert_sentences_to_ids(ja_sentences, word_to_id)

# padding処理
padded_ids = padding_and_truncate_sentence(sentence_ids)

# データをまとめた辞書を作成
processed_data = {
    'padded_ids': padded_ids,
    'labels': df['label_id'].tolist(),
    'word_to_id': word_to_id,
    'id_to_word': id_to_word,
    'label_to_id': label_to_id,
    'id_to_label': id_to_label,
}

# 学習用データはpickle形式にして保存する
with open('processed_data_maxlen512.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

print(f"保存ファイル: processed_data_maxlen512.pkl")
print(f"語彙数: {len(word_to_id)}")

# アルゴリズム選択（ベースラインモデル）

## ベースラインモデル設計

最初にCNNとGRUを1つのディープラーニングモデルの中に層（レイヤー）として組み込み、それぞれの長所を活かす**ハイブリッド**な構造を採用しました。

今回はLSTMではなく**GRU**を採用しました。

GRUはLSTMと比べてゲートの数が少なく構造がシンプルなため、**計算コストが低く学習が速い**傾向にあります。

それでいて、多くのタスクで**LSTMと同等**の性能を発揮することが知られています。

今回の課題では、**計算効率**と**実装の容易さ**を考慮し、RNN系手法としてGRUを採用しました。

CNNは入力された単語ベクトルの並びに対して、**局所的な特徴（n-gramのような短い単語の組み合わせ）**を抽出する役割を果たします。

例えば、「とても面白い」や「つまらない」といったキーフレーズを効率的に見つけ出す役割を担います。

CNNとGRUの組み合わせを選んだ理由としてこれらの手法の以下の特徴に着目しました：

* CNNの長所: 文中の重要なキーワードやフレーズ（局所的な特徴）を効率的に捉えることができる

* GRUの長所: RNN系列の手法として、単語の系列（シーケンス）の文脈や順序関係を効果的に捉えることができる

この2つを組み合わせることで、**「文章中の重要な部分（CNNが担当）が、どのような文脈で登場したか（GRUが担当）」**を同時に学習できるモデルを作ることができると考えました。

構築したニューラルネットワークは以下のような**4つの中間層**からなる構成となっています：

1. **Embedding層**: 単語をベクトルに変換する層
2. **CNN層** (1次元CNN): 特徴を抽出する層
3. **GRU層**: 系列情報を処理する再帰的な層
4. **全結合層**: 最終的な分類を行う層

## ベースラインモデル作成

### ベースラインモデル (CNN + GRU)

### ベースラインモデルの学習

### ベースラインモデルの評価 

### ベースラインモデルの考察

# アルゴリズム選択（改善モデル）

## 改善モデル設計

## 改善モデル作成

改善したニューラルネットワークは以下のような**4つの中間層**からなる構成となっています：

1. **Embedding層**: 単語をベクトルに変換する層
2. **Attentionメカニズム** (Self-Attention): 特徴を抽出する層
3. **GRU層**: 系列情報を処理する再帰的な層
4. **全結合層**: 最終的な分類を行う層

### 改善モデル（Self-Attention + GRU)

### 改善モデルの学習

### 改善モデルの評価

### 改善モデルの考察

# 考察と今後の課題

## 考察

## 今後の課題